In [1]:
from main import Task, train, test

In [2]:
task = Task()
samples, labels = task.next_batch()

In [3]:
# (batchsize, max_len + 3, vocab_size)
samples.shape

(100, 13, 3)

In [4]:
samples[0]

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [5]:
# (batchsize, 3, max_len + 1)
labels.shape

(100, 3, 11)

In [6]:
labels[0]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

# Without Multi-head Attention

In [7]:
_ = train(print_every=50, use_multihead=False)

Iteration 50 - Loss 1.8622273206710815
Iteration 100 - Loss 1.8322181701660156
Iteration 150 - Loss 1.8174924850463867
Iteration 200 - Loss 1.8192743062973022
Iteration 250 - Loss 1.74647057056427
Iteration 300 - Loss 1.7193609476089478
Iteration 350 - Loss 1.6702238321304321
Iteration 400 - Loss 1.6256766319274902
Iteration 450 - Loss 1.6380209922790527
Iteration 500 - Loss 1.5555511713027954
Iteration 550 - Loss 1.5122393369674683
Iteration 600 - Loss 1.4563356637954712
Iteration 650 - Loss 1.5420267581939697
Iteration 700 - Loss 1.4054031372070312
Iteration 750 - Loss 1.5382691621780396
Iteration 800 - Loss 1.5570237636566162
Iteration 850 - Loss 1.368138313293457
Iteration 900 - Loss 1.3676316738128662
Iteration 950 - Loss 1.1885355710983276
Iteration 1000 - Loss 1.3529635667800903
Iteration 1050 - Loss 1.28226900100708
Iteration 1100 - Loss 1.2506663799285889
Iteration 1150 - Loss 1.3605725765228271
Iteration 1200 - Loss 1.2255264520645142
Iteration 1250 - Loss 1.1979992389678955


# With Multi-head Attention

In [12]:
_ = train(print_every=50, use_multihead=True)

Iteration 50 - Loss 1.7857246398925781
Iteration 100 - Loss 1.735564112663269
Iteration 150 - Loss 1.868201732635498
Iteration 200 - Loss 1.772331714630127
Iteration 250 - Loss 1.8800278902053833
Iteration 300 - Loss 1.6627023220062256
Iteration 350 - Loss 1.760069727897644
Iteration 400 - Loss 1.609171986579895
Iteration 450 - Loss 1.5087788105010986
Iteration 500 - Loss 1.5319361686706543
Iteration 550 - Loss 1.5856661796569824
Iteration 600 - Loss 1.5295003652572632
Iteration 650 - Loss 1.3976054191589355
Iteration 700 - Loss 1.311308741569519
Iteration 750 - Loss 1.2208361625671387
Iteration 800 - Loss 1.186812162399292
Iteration 850 - Loss 1.0808887481689453
Iteration 900 - Loss 1.155795693397522
Iteration 950 - Loss 1.032659649848938
Iteration 1000 - Loss 0.9723690748214722
Iteration 1050 - Loss 0.96322101354599
Iteration 1100 - Loss 0.9177731275558472
Iteration 1150 - Loss 0.9020969867706299
Iteration 1200 - Loss 0.8114311099052429
Iteration 1250 - Loss 0.8514971137046814
Iterat

In [13]:
test()


Input: 
[[['A']
  ['C']
  ['C']
  ['C']
  ['A']
  ['B']
  ['B']
  ['B']
  ['C']
  ['A']
  ['B']
  ['C']
  ['C']]]

Prediction: 
[[2 4 4]]

Encoder-Decoder Attention: 
Head #0
	Attention of Output step 0 to Input steps
	[0.0, 0.059, 0.104, 0.001, 0.362, 0.032, 0.032, 0.032, 0.001, 0.343, 0.031, 0.001, 0.001]
	Attention of Output step 1 to Input steps
	[0.001, 0.834, 0.001, 0.002, 0.05, 0.014, 0.014, 0.014, 0.002, 0.05, 0.014, 0.002, 0.002]
	Attention of Output step 2 to Input steps
	[0.0, 0.0, 0.827, 0.003, 0.057, 0.013, 0.013, 0.013, 0.003, 0.052, 0.013, 0.003, 0.003]

L2-Norm of Input Positional Encoding:
[2.077, 2.003, 2.008, 0.565, 0.565, 0.568, 0.564, 0.562, 0.566, 0.569, 0.567, 0.57, 0.561]
